This notebook contains some intermediate steps in computing the Bahadur efficiency, slopes using Sympy.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import kgof.data as data
import kgof.glo as glo
import kgof.goftest as gof
import kgof.kernel as kernel
import kgof.plot as plot
import kgof.util as util

import scipy.stats as stats
import sympy as smp
# http://docs.sympy.org/dev/modules/physics/vector/api/printing.html
smp.init_printing(use_latex='mathjax', latex_mode='inline')

In [ ]:
# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 16
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

## Product of 2 Gaussian distributions

In [ ]:
mp, mq, sp, sq = smp.symbols(r'mu_p mu_q sigma^2_p sigma^2_q')
s = 1/(1/sp + 1/sq)
b = mp/sp + mq/sq
c = mp**2/sp + mq**2/sq

In [ ]:
-b**2*s + c

In [ ]:
smp.simplify(-b**2*s + c)

In [ ]:
# mean of the product of 2 Gaussians
smp.simplify(s*b)

## Assume d=1, J=1, p,q=Gaussian with different parameters

In [ ]:
# computation of E_q \xi(x, v)
v, mq, mp = smp.symbols(r'v mu_q mu_p')
sk = smp.Symbol('s_k', positive=True)
sp = smp.Symbol('s_p', positive=True)
sq = smp.Symbol('s_q', positive=True)

s = 1/(1/sk + 1/sq)
mu = (v/sk+mq/sq)*s

In [ ]:
exi = -smp.sqrt(s/sq)*smp.exp(-(v-mq)**2/(sk+sq))*((mu-mp)/sp + (mu-v)/sk)

In [ ]:
smp.simplify(exi)

In [ ]:
smp.simplify(exi.subs({mp: mq}))

In [ ]:
smp.simplify(exi.subs({sq: sp}))

In [ ]:
smp.simplify(exi.subs({sq: sp, mp: mq}))

### Compute $\boldsymbol{\Sigma}=\mathbb{V}_{\mathbf{x} \sim p}[\boldsymbol{\tau}(\mathbf{x})]$ when $d=1, J=1$ under $H_0$. Note $p=q$.

In [ ]:
x = smp.symbols('x')
alpha = (x-mp)/sp + (x-v)/sk
# mean of the Gaussian
m = (2*v/sk + mq/sq)/(2/sk+1/sq)
s = 1/(2/sk+1/sq)

In [ ]:
m

In [ ]:
a2 = smp.collect(smp.expand(alpha**2, x), x)
a2

In [ ]:
ealpha2 = mp**2/sp**2 + 2*mp*v/(sk*sp) + v**2/sk**2 + (s+m**2)*(1/sp**2 + 2/(sk*sp) + 1/sk**2) + \
    m*(-2*mp/sp**2 -2*mp/(sk*sp) -2*v/(sk*sp) -2*v/sk**2)
ealpha2

In [ ]:
smp.simplify(smp.together(ealpha2))

In [ ]:
exi2 = smp.sqrt(s/sq)*smp.exp(-(v-mq)**2/(sk/2+sq))*ealpha2
exi2

In [ ]:
#var_xi = exi2 - exi**2 # Don't need -exi**2 under H0
var_xi = exi2
pp, pq, pk = smp.symbols('tau_p tau_q tau_k', positive=True)
var_xi_prec = var_xi.subs({sq: 1/pq, sp: 1/pp, sk: 1/pk})

In [ ]:
# Make p=q i.e., make the means the same, and the variances (precisions) the same
var_xi_prec_h0 = var_xi_prec.subs({pq: pp, mq: mp})
smp.simplify(var_xi_prec_h0)

## Bahadur slope

In [ ]:
fssd2 = (exi**2).subs({sq: 1/pq, sp: 1/pp, sk: 1/pk})
fssd2

In [ ]:
var_xi_prec_h0

In [ ]:
slope = smp.simplify(fssd2)/var_xi_prec_h0
slope

In [ ]:
smp.simplify(slope)

In [ ]:
slope_standard = smp.simplify(slope.subs({mp: 0, pp:1, pk: 1, pq: 1/sq}))
slope_standard

In [ ]:
# mqs = [-1, 0, 1.2]
mqs = [0]
sqs = [0.05, 1, 1.8]
# sqs = [1]
vs = np.linspace(-8, 8, 300)
plt.figure(figsize=(8, 6))
for mq1 in mqs:
    for sq1 in sqs:
        f = smp.lambdify(v, slope_standard.subs({mq: mq1, sq: sq1}))
        fs = [f(tmp) for tmp in vs]
        plt.plot(vs, fs, label=r'$m_q: {0}, s_q: {1}$'.format(mq1, sq1))
    
plt.legend(bbox_to_anchor=(1.6, 1))
plt.xlabel('v')
plt.ylabel('slope')